# Segmenting_And_Clustering_Neighborhoods_In_Toronto


## 1. scrape the Wikipedia page to pandas dataframe

**Extract Table from wikipedia**

In [170]:
# Extract table from wikipedia
from IPython.core.display import HTML
from pandas.io.html import read_html
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikitables = read_html(page, attrs={"class": "wikitable"})
print("Extracted {num} wikitables".format(num = len(wikitables)))

Extracted 1 wikitables


In [171]:
df_toronto = wikitables[0]
df_toronto.head()

Postal code           Borough                Neighborhood
0         M1A      Not assigned                         NaN
1         M2A      Not assigned                         NaN
2         M3A        North York                   Parkwoods
3         M4A        North York            Victoria Village
4         M5A  Downtown Toronto  Regent Park / Harbourfront

**Ignore cells with a borough that is Not assigned**

In [172]:
# Ignore cells with a borough that is Not assigned 
df_toronto.drop(df_toronto[df_toronto.Borough == "Not assigned"].index, inplace = True)
df_toronto.head()

Postal code           Borough                                  Neighborhood
2         M3A        North York                                     Parkwoods
3         M4A        North York                              Victoria Village
4         M5A  Downtown Toronto                    Regent Park / Harbourfront
5         M6A        North York             Lawrence Manor / Lawrence Heights
6         M7A  Downtown Toronto  Queen's Park / Ontario Provincial Government

**Replace '/' by cammas**

In [173]:
# replace / by comma
df_toronto['Neighborhood'] = df_toronto['Neighborhood'].str.replace(' / ', ', ')
df_toronto.head()

Postal code           Borough                                 Neighborhood
2         M3A        North York                                    Parkwoods
3         M4A        North York                             Victoria Village
4         M5A  Downtown Toronto                    Regent Park, Harbourfront
5         M6A        North York             Lawrence Manor, Lawrence Heights
6         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

**The neighborhood will be the same as the borough, when the cell has a borough but a Not assigned neighborhood**

In [174]:
df_toronto.loc[df_toronto['Neighborhood'] == "Not assigned", 'Neighborhood'] = df_toronto['Borough']

**Number of rows in the dataframe**

In [175]:
df_toronto.shape[0]

103

# 2.  Get the latitude and the longitude coordinates of each neighborhood

In [176]:
import numpy as np
from geopy.geocoders import Nominatim
import folium
import requests 
import json
from pandas.io.json import json_normalize
# Add Latitude and Longitude columns
#df_toronto['Latitude'] = np.nan
#df_toronto['Longitude'] = np.nan
#iterate dataframe and add locations
#for Borough, Neighborhood in zip(df_toronto.Borough, df_toronto.Neighborhood):
#    address = '{}, {}'.format(Borough, Neighborhood)
#    geolocator = Nominatim(user_agent="ny_explorer")
#    location = geolocator.geocode(address)
#    df_toronto['Latitude'] = location.latitude
#    df_toronto['Longitude'] = location.longitude

In [177]:
import pandas as pd
df_coor = pd.read_csv('Geospatial_Coordinates.csv')
df_coor.rename(columns={"Postal Code": "Postal code"}, inplace=True)

In [178]:
df_toronto = pd.merge(df_toronto,df_coor,on='Postal code')

In [179]:
df_toronto.head(15)

Postal code           Borough  \
0          M3A        North York   
1          M4A        North York   
2          M5A  Downtown Toronto   
3          M6A        North York   
4          M7A  Downtown Toronto   
5          M9A         Etobicoke   
6          M1B       Scarborough   
7          M3B        North York   
8          M4B         East York   
9          M5B  Downtown Toronto   
10         M6B        North York   
11         M9B         Etobicoke   
12         M1C       Scarborough   
13         M3C        North York   
14         M4C         East York   

                                         Neighborhood   Latitude  Longitude  
0                                           Parkwoods  43.753259 -79.329656  
1                                    Victoria Village  43.725882 -79.315572  
2                           Regent Park, Harbourfront  43.654260 -79.360636  
3                    Lawrence Manor, Lawrence Heights  43.718518 -79.464763  
4         Queen's Park, Ontario Provincial Government  43.662301 -79.389494  
5                                    Islington Avenue  43.667856 -79.532242  
6                                      Malvern, Rouge  43.806686 -79.194353  
7                                           Don Mills  43.745906 -79.352188  
8                     Parkview Hill, Woodbine Gardens  43.706397 -79.309937  
9                            Garden District, Ryerson  43.657162 -79.378937  
10                                          Glencairn  43.709577 -79.445073  
11  West Deane Park, Princess Gardens, Martin Grov...  43.650943 -79.554724  
12             Rouge Hill, Port Union, Highland Creek  43.784535 -79.160497  
13                                          Don Mills  43.725900 -79.340923  
14                                   Woodbine Heights  43.695344 -79.318389

# 3. Explore and cluster the neighborhoods in Toronto

**Create a map of Toronto with neighborhoods superimposed on top**

In [180]:
# create map of Toronto using latitude and longitude values
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode('Toronto')
t_latitude = location.latitude
t_longitude = location.longitude
map_toronto = folium.Map(location=[t_latitude, t_longitude], zoom_start=10)

# add markers to map
for borough, neighborhood, lat, lng  in zip(df_toronto['Borough'], df_toronto['Neighborhood'], df_toronto['Latitude'], df_toronto['Longitude']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
HTML(map_toronto._repr_html_())

However, for illustration purposes, let's simplify the above map and segment and cluster only the boroughs that contain the word Toronto. So let's slice the original dataframe and create a new dataframe of the Toronto data.

In [181]:
toronto_data = df_toronto[df_toronto['Borough'].str.contains("Toronto")]
toronto_data.head()

Postal code           Borough                                 Neighborhood  \
2          M5A  Downtown Toronto                    Regent Park, Harbourfront   
4          M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
9          M5B  Downtown Toronto                     Garden District, Ryerson   
15         M5C  Downtown Toronto                               St. James Town   
19         M4E      East Toronto                                  The Beaches   

     Latitude  Longitude  
2   43.654260 -79.360636  
4   43.662301 -79.389494  
9   43.657162 -79.378937  
15  43.651494 -79.375418  
19  43.676357 -79.293031

As we did with all of Toronto, let's visualizat the neighborhoods.

In [182]:
latitude = 43.6739000
longitude = -79.3839347
map_min_Toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
# add markers to map
# add markers to map
for borough, neighborhood, lat, lng  in zip(toronto_data['Borough'], toronto_data['Neighborhood'], toronto_data['Latitude'], toronto_data['Longitude']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_min_Toronto)  
    
HTML(map_min_Toronto._repr_html_())

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [183]:
CLIENT_ID = 'SKNCUUXV2LVE2LU5BDEPEJ4V1ITSVSK0PZX1NCTILYXPKNVF' # your Foursquare ID
CLIENT_SECRET = 'CJIAFH4GL4KVPDC5HB143DBZLDDFPJTV0F0EQDFZRVW5WP0C' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### Let's explore one neighborhood in our dataframe.

In [184]:
toronto_data.loc[19, 'Neighborhood']

'The Beaches'

Get the neighborhood's latitude and longitude values.

In [185]:
neighborhood_latitude = toronto_data.loc[19, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[19, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[19, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


#### Now, let's get the top 100 venues that are in The Beaches within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [194]:
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=SKNCUUXV2LVE2LU5BDEPEJ4V1ITSVSK0PZX1NCTILYXPKNVF&client_secret=CJIAFH4GL4KVPDC5HB143DBZLDDFPJTV0F0EQDFZRVW5WP0C&ll=43.67635739999999,-79.2930312&v=20180605&radius=500&limit=100'

Send the GET request and examine the resutls

In [195]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e977d5729ce6a001b9fc85c'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [196]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [197]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

name         categories        lat        lng
0                   Glen Manor Ravine              Trail  43.676821 -79.293942
1  The Big Carrot Natural Food Market  Health Food Store  43.678879 -79.297734
2                 Grover Pub and Grub                Pub  43.679181 -79.297215
3                       Upper Beaches       Neighborhood  43.680563 -79.292869

And how many venues were returned by Foursquare?

In [198]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


### Explore Neighborhoods in Borough that contains the world Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [199]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *Toronto_venues*.

In [201]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst  Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town

#### Let's check the size of the resulting dataframe

In [202]:
print(toronto_venues.shape)
toronto_venues.head()

(1627, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Regent Park, Harbourfront               43.65426              -79.360636   
1  Regent Park, Harbourfront               43.65426              -79.360636   
2  Regent Park, Harbourfront               43.65426              -79.360636   
3  Regent Park, Harbourfront               43.65426              -79.360636   
4  Regent Park, Harbourfront               43.65426              -79.360636   

                    Venue  Venue Latitude  Venue Longitude  \
0        Roselle Desserts       43.653447       -79.362017   
1           Tandem Coffee       43.653559       -79.361809   
2  Cooper Koo Family YMCA       43.653249       -79.358008   
3     Body Blitz Spa East       43.654735       -79.359874   
4      Morning Glory Cafe       43.653947       -79.361149   

        Venue Category  
0               Bakery  
1          Coffee Shop  
2  Distribution Center  
3                  Spa  
4       Breakfast Spot

Let's check how many venues were returned for each neighborhood

In [203]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            55   
Brockton, Parkdale Village, Exhibition Place                           22   
Business reply mail Processing CentrE                                  17   
CN Tower, King and Spadina, Railway Lands, Harb...                     18   
Central Bay Street                                                     64   
Christie                                                               18   
Church and Wellesley                                                   75   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             36   
Davisville North                                                        8   
Dufferin, Dovercourt Village                                           17   
First Canadian Place, Underground city                                100   
Forest Hill North & West                                                4   
Garden District, Ryerson                                              100   
Harbourfront East, Union Station, Toronto Islands                     100   
High Park, The Junction South                                          24   
India Bazaar, The Beaches West                                         20   
Kensington Market, Chinatown, Grange Park                              62   
Lawrence Park                                                           4   
Little Portugal, Trinity                                               43   
Moore Park, Summerhill East                                             3   
North Toronto West                                                     20   
Parkdale, Roncesvalles                                                 14   
Queen's Park, Ontario Provincial Government                            33   
Regent Park, Harbourfront                                              47   
Richmond, Adelaide, King                                               97   
Rosedale                                                                4   
Roselawn                                                                1   
Runnymede, Swansea                                                     36   
St. James Town                                                         85   
St. James Town, Cabbagetown                                            46   
Stn A PO Boxes                                                         95   
Studio District                                                        40   
Summerhill West, Rathnelly, South Hill, Forest ...                     16   
The Annex, North Midtown, Yorkville                                    22   
The Beaches                                                             4   
The Danforth West, Riverdale                                           43   
Toronto Dominion Centre, Design Exchange                              100   
University of Toronto, Harbord                                         34   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             55   
Brockton, Parkdale Village, Exhibition Place                            22   
Business reply mail Processing CentrE                                   17   
CN Tower, King and Spadina, Railway Lands, Harb...                      18   
Central Bay Street                                                      64   
Christie                                                                18   
Church and Wellesley                                                    75   
Commerce Court, Victoria Hotel                                         100   
Davisville                                                              36   
Davisville North                    

Let's find out how many unique categories can be curated from all the returned venues

In [204]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 226 uniques categories.


### Analyze Each Neighborhood

In [205]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  Airport  Airport Food Court  Airport Gate  Airport Lounge  \
0            0        0                   0             0               0   
1            0        0                   0             0               0   
2            0        0                   0             0               0   
3            0        0                   0             0               0   
4            0        0                   0             0               0   

   Airport Service  Airport Terminal  American Restaurant  Antique Shop  \
0                0                 0                    0             0   
1                0                 0                    0             0   
2                0                 0                    0             0   
3                0                 0                    0             0   
4                0                 0                    0             0   

   Aquarium  ...  Theme Restaurant  Toy / Game Store  Trail  Train Station  \
0         0  ...                 0                 0      0              0   
1         0  ...                 0                 0      0              0   
2         0  ...                 0                 0      0              0   
3         0  ...                 0                 0      0              0   
4         0  ...                 0                 0      0              0   

   Vegetarian / Vegan Restaurant  Video Game Store  Vietnamese Restaurant  \
0                              0                 0                      0   
1                              0                 0                      0   
2                              0                 0                      0   
3                              0                 0                      0   
4                              0                 0                      0   

   Wine Bar  Wine Shop  Women's Store  
0         0          0              0  
1         0          0              0  
2         0          0              0  
3         0          0              0  
4         0          0              0  

[5 rows x 226 columns]

And let's examine the new dataframe size.

In [206]:
toronto_onehot.shape

(1627, 226)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [207]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Yoga Studio   Airport  \
0                                         Berczy Park     0.000000  0.000000   
1        Brockton, Parkdale Village, Exhibition Place     0.000000  0.000000   
2               Business reply mail Processing CentrE     0.000000  0.000000   
3   CN Tower, King and Spadina, Railway Lands, Har...     0.000000  0.055556   
4                                  Central Bay Street     0.015625  0.000000   
5                                            Christie     0.000000  0.000000   
6                                Church and Wellesley     0.026667  0.000000   
7                      Commerce Court, Victoria Hotel     0.000000  0.000000   
8                                          Davisville     0.000000  0.000000   
9                                    Davisville North     0.000000  0.000000   
10                       Dufferin, Dovercourt Village     0.000000  0.000000   
11             First Canadian Place, Underground city     0.000000  0.000000   
12                           Forest Hill North & West     0.000000  0.000000   
13                           Garden District, Ryerson     0.000000  0.000000   
14  Harbourfront East, Union Station, Toronto Islands     0.000000  0.000000   
15                      High Park, The Junction South     0.000000  0.000000   
16                     India Bazaar, The Beaches West     0.000000  0.000000   
17          Kensington Market, Chinatown, Grange Park     0.000000  0.000000   
18                                      Lawrence Park     0.000000  0.000000   
19                           Little Portugal, Trinity     0.023256  0.000000   
20                        Moore Park, Summerhill East     0.000000  0.000000   
21                                 North Toronto West     0.050000  0.000000   
22                             Parkdale, Roncesvalles     0.000000  0.000000   
23        Queen's Park, Ontario Provincial Government     0.030303  0.000000   
24                          Regent Park, Harbourfront     0.021277  0.000000   
25                           Richmond, Adelaide, King     0.000000  0.000000   
26                                           Rosedale     0.000000  0.000000   
27                                           Roselawn     0.000000  0.000000   
28                                 Runnymede, Swansea     0.000000  0.000000   
29                                     St. James Town     0.000000  0.000000   
30                        St. James Town, Cabbagetown     0.000000  0.000000   
31                                     Stn A PO Boxes     0.010526  0.000000   
32                                    Studio District     0.025000  0.000000   
33  Summerhill West, Rathnelly, South Hill, Forest...     0.000000  0.000000   
34                The Annex, North Midtown, Yorkville     0.000000  0.000000   
35                                        The Beaches     0.000000  0.000000   
36                       The Danforth West, Riverdale     0.023256  0.000000   
37           Toronto Dominion Centre, Design Exchange     0.000000  0.000000   
38                     University of Toronto, Harbord     0.029412  0.000000   

    Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0             0.000000      0.000000        0.000000         0.000000   
1             0.000000      0.000000        0.000000         0.000000   
2             0.000000      0.000000        0.000000         0.000000   
3             0.055556      0.055556        0.111111         0.166667   
4             0.000000      0.000000        0.000000         0.000000   
5             0.000000      0.000000        0.000000         0.000000   
6             0.000000      0.000000        0.000000         0.000000   
7             0.000000      0.000000        0.000000         0.000000   
8             0.000000      0.000000        0.000000         0.000000   
9             0.000000      0.000000        0.000000         0.000000   
10            0.000000      0.000000 

#### Let's confirm the new size

In [208]:
toronto_grouped.shape

(39, 226)

#### Let's print each neighborhood along with the top 5 most common venues

In [209]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.05
1        Cocktail Bar  0.04
2  Seafood Restaurant  0.04
3  Italian Restaurant  0.04
4         Cheese Shop  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.14
1     Coffee Shop  0.09
2  Breakfast Spot  0.09
3   Grocery Store  0.05
4          Bakery  0.05


----Business reply mail Processing CentrE----
           venue  freq
0            Spa  0.06
1  Auto Workshop  0.06
2     Smoke Shop  0.06
3           Park  0.06
4  Burrito Place  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst  Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.17
1    Airport Lounge  0.11
2  Airport Terminal  0.11
3   Harbor / Marina  0.06
4       Coffee Shop  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.19
1  Italian Restaurant  0.06
2                Café  0.05
3      Sandw

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [210]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [211]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1       Brockton, Parkdale Village, Exhibition Place                  Café   
2              Business reply mail Processing CentrE                  Park   
3  CN Tower, King and Spadina, Railway Lands, Har...       Airport Service   
4                                 Central Bay Street           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0           Cheese Shop    Seafood Restaurant                Bakery   
1        Breakfast Spot           Coffee Shop         Grocery Store   
2         Auto Workshop            Comic Shop           Pizza Place   
3        Airport Lounge      Airport Terminal       Harbor / Marina   
4    Italian Restaurant                  Café        Sandwich Place   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0            Restaurant    Italian Restaurant              Beer Bar   
1          Intersection                   Bar               Stadium   
2               Butcher            Restaurant         Burrito Place   
3               Airport    Airport Food Court          Airport Gate   
4                   Spa       Bubble Tea Shop        Ice Cream Shop   

       8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0                       Café        Farmers Market           Cocktail Bar  
1                     Bakery    Italian Restaurant           Climbing Gym  
2                    Brewery    Light Rail Station             Skate Park  
3                        Bar   Rental Car Location               Boutique  
4  Middle Eastern Restaurant      Sushi Restaurant    Japanese Restaurant

### Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [212]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 1, 2, 2, 2, 2, 2, 1, 1])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [213]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

Postal code           Borough                                 Neighborhood  \
2          M5A  Downtown Toronto                    Regent Park, Harbourfront   
4          M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
9          M5B  Downtown Toronto                     Garden District, Ryerson   
15         M5C  Downtown Toronto                               St. James Town   
19         M4E      East Toronto                                  The Beaches   

     Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
2   43.654260 -79.360636               2           Coffee Shop   
4   43.662301 -79.389494               2           Coffee Shop   
9   43.657162 -79.378937               2           Coffee Shop   
15  43.651494 -79.375418               2                  Café   
19  43.676357 -79.293031               0                 Trail   

   2nd Most Common Venue 3rd Most Common Venue      4th Most Common Venue  \
2                   Park                Bakery                        Pub   
4                  Diner           Yoga Studio                   Creperie   
9         Clothing Store                  Café  Middle Eastern Restaurant   
15           Coffee Shop          Cocktail Bar                 Restaurant   
19     Health Food Store                   Pub              Women's Store   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
2                Theater    Mexican Restaurant                  Café   
4               Beer Bar        Sandwich Place          Burger Joint   
9     Italian Restaurant       Bubble Tea Shop        Cosmetics Shop   
15              Beer Bar                 Hotel   American Restaurant   
19      Department Store  Ethiopian Restaurant     Electronics Store   

          8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
2                Breakfast Spot            Shoe Store         Farmers Market  
4                 Burrito Place            Restaurant                   Café  
9           Japanese Restaurant                 Plaza                  Diner  
15               Lingerie Store   Japanese Restaurant     Seafood Restaurant  
19  Eastern European Restaurant   Dumpling Restaurant             Donut Shop

Finally, let's visualize the resulting clusters

In [214]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
latitude = 43.6739000
longitude = -79.3839347
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
HTML(map_clusters._repr_html_())

### Examine Clusters

Examine each cluster and determine the discriminating venue categories that distinguish each cluster.

#### Cluster 1

In [215]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
19  East Toronto               0                 Trail     Health Food Store   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
19                   Pub         Women's Store      Department Store   

   6th Most Common Venue 7th Most Common Venue        8th Most Common Venue  \
19  Ethiopian Restaurant     Electronics Store  Eastern European Restaurant   

   9th Most Common Venue 10th Most Common Venue  
19   Dumpling Restaurant             Donut Shop

#### Cluster 2

In [216]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
47      East Toronto               1        Sandwich Place   
61   Central Toronto               1                  Park   
67   Central Toronto               1                  Park   
74   Central Toronto               1        Sandwich Place   
79   Central Toronto               1        Sandwich Place   
100     East Toronto               1                  Park   

    2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
47   Fast Food Restaurant                   Pub     Food & Drink Shop   
61                 Lawyer           Swim School              Bus Line   
67      Food & Drink Shop        Breakfast Spot                   Gym   
74                   Café           Coffee Shop                  Park   
79           Dessert Shop           Pizza Place                   Gym   
100         Auto Workshop            Comic Shop           Pizza Place   

    5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
47           Liquor Store         Burrito Place            Restaurant   
61            Event Space  Ethiopian Restaurant     Electronics Store   
67   Gym / Fitness Center                 Hotel      Department Store   
74            Pizza Place          Liquor Store          Burger Joint   
79     Italian Restaurant                  Café      Sushi Restaurant   
100               Butcher            Restaurant         Burrito Place   

           8th Most Common Venue      9th Most Common Venue  \
47            Italian Restaurant                 Steakhouse   
61   Eastern European Restaurant        Dumpling Restaurant   
67                Sandwich Place                      Diner   
74             Indian Restaurant  Middle Eastern Restaurant   
79                   Coffee Shop           Greek Restaurant   
100                      Brewery         Light Rail Station   

    10th Most Common Venue  
47       Fish & Chips Shop  
61              Donut Shop  
67          Discount Store  
74                     Pub  
79        Toy / Game Store  
100             Skate Park

#### Cluster 3

In [217]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
2   Downtown Toronto               2           Coffee Shop   
4   Downtown Toronto               2           Coffee Shop   
9   Downtown Toronto               2           Coffee Shop   
15  Downtown Toronto               2                  Café   
20  Downtown Toronto               2           Coffee Shop   
24  Downtown Toronto               2           Coffee Shop   
25  Downtown Toronto               2         Grocery Store   
30  Downtown Toronto               2           Coffee Shop   
31      West Toronto               2                Bakery   
36  Downtown Toronto               2           Coffee Shop   
37      West Toronto               2                   Bar   
41      East Toronto               2      Greek Restaurant   
42  Downtown Toronto               2           Coffee Shop   
43      West Toronto               2                  Café   
48  Downtown Toronto               2           Coffee Shop   
54      East Toronto               2                  Café   
69      West Toronto               2    Mexican Restaurant   
73   Central Toronto               2        Clothing Store   
75      West Toronto               2             Gift Shop   
80  Downtown Toronto               2                  Café   
81      West Toronto               2                  Café   
84  Downtown Toronto               2                  Café   
86   Central Toronto               2                   Pub   
87  Downtown Toronto               2       Airport Service   
92  Downtown Toronto               2           Coffee Shop   
96  Downtown Toronto               2           Coffee Shop   
97  Downtown Toronto               2           Coffee Shop   
99  Downtown Toronto               2           Coffee Shop   

   2nd Most Common Venue          3rd Most Common Venue  \
2                   Park                         Bakery   
4                  Diner                    Yoga Studio   
9         Clothing Store                           Café   
15           Coffee Shop                   Cocktail Bar   
20           Cheese Shop             Seafood Restaurant   
24    Italian Restaurant                           Café   
25                  Café                           Park   
30                  Café                     Restaurant   
31              Pharmacy                  Grocery Store   
36              Aquarium                          Hotel   
37            Restaurant  Vegetarian / Vegan Restaurant   
41    Italian Restaurant                    Coffee Shop   
42                 Hotel                           Café   
43        Breakfast Spot                    Coffee Shop   
48                  Café                     Restaurant   
54           Coffee Shop                      Gastropub   
69                  Café                Thai Restaurant   
73           Coffee Shop                    Yoga Studio   
75             Bookstore                  Movie Theater   
80            Restaurant                            Bar   
81           Coffee Shop             Italian Restaurant   
84           Coffee Shop             Mexican Restaurant   
86           Coffee Shop                     Restaurant   
87        Airport Lounge               Airport Terminal   
92            Restaurant             Italian Restaurant   
96                Bakery                           Café   
97                  Café                     Restaurant   
99   Japanese Restaurant                        Gay Bar   

        4th Most Common Venue 5th Most Common Venue  \
2                         Pub               Theater   
4                    Creperie              Beer Bar   
9   Middle Eastern Restaurant    Italian Restaurant   
15                 Restaurant              Beer Bar   
20                     Bakery            Restaurant   
24             Sandwich Place                   Spa   
25         Italian Restaurant           Candy Store   
30                        Gym                 Hotel   
31     

#### Cluster 4

In [218]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
62  Central Toronto               3                Garden   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
62         Women's Store      Department Store  Ethiopian Restaurant   

   5th Most Common Venue        6th Most Common Venue 7th Most Common Venue  \
62     Electronics Store  Eastern European Restaurant   Dumpling Restaurant   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
62            Donut Shop      Doner Restaurant                Dog Run

#### Cluster 5

In [219]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
68   Central Toronto               4                  Park   
83   Central Toronto               4                  Park   
91  Downtown Toronto               4                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
68         Jewelry Store                 Trail      Sushi Restaurant   
83            Playground          Tennis Court         Deli / Bodega   
91            Playground                 Trail         Deli / Bodega   

   5th Most Common Venue 6th Most Common Venue        7th Most Common Venue  \
68         Women's Store      Department Store            Electronics Store   
83  Ethiopian Restaurant     Electronics Store  Eastern European Restaurant   
91  Ethiopian Restaurant     Electronics Store  Eastern European Restaurant   

          8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
68  Eastern European Restaurant   Dumpling Restaurant             Donut Shop  
83          Dumpling Restaurant            Donut Shop       Doner Restaurant  
91          Dumpling Restaurant            Donut Shop       Doner Restaurant